In [24]:
import re
import pandas as pd
import mysql.connector
import warnings
from IPython.display import display
import csv
#import matplotlib.pyplot as plt
# Suppress a specific warning
warnings.filterwarnings("ignore", category=UserWarning)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

datamovesoutput = []
datamovesactivityoutput=[]
host = "localhost" 
user = "moves"
password = "moves"
port=3306
vehicle_type = 'refusetruck'
year = '2024'

In [25]:
def assign_vehicle_value(category):
    if category == 'combinationlonghaultruck':
        return 62
    elif category == 'combinationshorthaultruck':
        return 61
    elif category == 'lightcommercialtruck':
        return 32
    elif category == 'motorcycle':
        return 11
    elif category == 'motorhome':
        return 54
    elif category == 'otherbuses':
        return 41
    elif category == 'passengecar':
        return 21
    elif category == 'passengertruck':
        return 31
    elif category == 'refusetruck':
        return 51
    elif category == 'schoolbus':
        return 43
    elif category == 'singleunitlonghaultruck':
        return 53
    elif category == 'singleunitshorthaultruck':
        return 52
    elif category == 'transitbus':
        return 42
    else:
        return 0

In [26]:
connection= mysql.connector.connect(host=host,user=user, password=password, port=port,use_pure=True)
cursor=connection.cursor()
pattern_database = "%2024age%speed%opdb"
query = f"SHOW DATABASES LIKE '{pattern_database}'"
cursor.execute(query)

# Fetch all the matching database names
databases = cursor.fetchall()
database_array = [database[0] for database in databases]
print(database_array)

cursor.close()
connection.close()

['otherbuses2024age0speed10opdb', 'otherbuses2024age0speed11opdb', 'otherbuses2024age0speed12opdb', 'otherbuses2024age0speed13opdb', 'otherbuses2024age0speed14opdb', 'otherbuses2024age0speed15opdb', 'otherbuses2024age0speed16opdb', 'otherbuses2024age0speed1opdb', 'otherbuses2024age0speed2opdb', 'otherbuses2024age0speed3opdb', 'otherbuses2024age0speed4opdb', 'otherbuses2024age0speed5opdb', 'otherbuses2024age0speed6opdb', 'otherbuses2024age0speed7opdb', 'otherbuses2024age0speed8opdb', 'otherbuses2024age0speed9opdb', 'otherbuses2024age10speed10opdb', 'otherbuses2024age10speed11opdb', 'otherbuses2024age10speed12opdb', 'otherbuses2024age10speed13opdb', 'otherbuses2024age10speed14opdb', 'otherbuses2024age10speed15opdb', 'otherbuses2024age10speed16opdb', 'otherbuses2024age10speed1opdb', 'otherbuses2024age10speed2opdb', 'otherbuses2024age10speed3opdb', 'otherbuses2024age10speed4opdb', 'otherbuses2024age10speed5opdb', 'otherbuses2024age10speed6opdb', 'otherbuses2024age10speed7opdb', 'otherbuses

In [27]:
for database in database_array:
    databaseConnection= mysql.connector.connect(host=host,user=user, password=password,database=database,port=port,use_pure=True)
    print(database)
    pattern = r"(.*)2024.*?age(\d+).*?speed(\d+)"
    matches = re.findall(pattern, database)

    if matches:
        vehicletype, age, speed = matches[0] 
    vehicleid=assign_vehicle_value(vehicletype);
    query1 ="""SELECT SUM(emissionQuant)AS emission,pollutantID,fuelTypeID from movesoutput  where pollutantID IN (90,3,110,116,117,91) GROUP  BY pollutantID,fuelTypeID ORDER BY fueltypeid"""
    query2 = """SELECT SUM(activity) AS distance, fuelTypeID FROM movesactivityoutput group by fuelTypeID"""
    dfmovesoutput = pd.read_sql(query1,databaseConnection)
    dfmovesactivityoutput= pd.read_sql(query2,databaseConnection)
    dfmovesoutput['VehicleID']=vehicleid
    dfmovesoutput['speed']= speed
    dfmovesactivityoutput['speed']= speed
    dfmovesoutput['age']= age
    dfmovesactivityoutput['age']= age
    dfmovesactivityoutput['VehicleID']=vehicleid
    datamovesoutput.append(dfmovesoutput)
    datamovesactivityoutput.append(dfmovesactivityoutput)
databaseConnection.close()
fulldatamovesoutput = pd.concat(datamovesoutput)
fulldatamovesactivityoutput=pd.concat(datamovesactivityoutput)
merged_df = pd.merge(fulldatamovesoutput, fulldatamovesactivityoutput, on=['age', 'speed', 'fuelTypeID','VehicleID'], how='left')
# Replace NaN values in the 'distance' column with 0
merged_df['distance'].fillna(0, inplace=True)

# Divide emission by distance
merged_df['emission_per_distance'] = merged_df['emission'] / merged_df['distance']

# Replace infinite values (caused by division by zero) with 0
merged_df['emission_per_distance'].fillna(0, inplace=True)

otherbuses2024age0speed10opdb
otherbuses2024age0speed11opdb
otherbuses2024age0speed12opdb
otherbuses2024age0speed13opdb
otherbuses2024age0speed14opdb
otherbuses2024age0speed15opdb
otherbuses2024age0speed16opdb
otherbuses2024age0speed1opdb
otherbuses2024age0speed2opdb
otherbuses2024age0speed3opdb
otherbuses2024age0speed4opdb
otherbuses2024age0speed5opdb
otherbuses2024age0speed6opdb
otherbuses2024age0speed7opdb
otherbuses2024age0speed8opdb
otherbuses2024age0speed9opdb
otherbuses2024age10speed10opdb
otherbuses2024age10speed11opdb
otherbuses2024age10speed12opdb
otherbuses2024age10speed13opdb
otherbuses2024age10speed14opdb
otherbuses2024age10speed15opdb
otherbuses2024age10speed16opdb
otherbuses2024age10speed1opdb
otherbuses2024age10speed2opdb
otherbuses2024age10speed3opdb
otherbuses2024age10speed4opdb
otherbuses2024age10speed5opdb
otherbuses2024age10speed6opdb
otherbuses2024age10speed7opdb
otherbuses2024age10speed8opdb
otherbuses2024age10speed9opdb
otherbuses2024age11speed10opdb
otherbuses2

C:\Users\skanigir\AppData\Local\Temp\ipykernel_147632\1309033102.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['distance'].fillna(0, inplace=True)
C:\Users\skanigir\AppData\Local\Temp\ipykernel_147632\1309033102.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.



In [28]:
fulldatamovesactivityoutput

distance  fuelTypeID speed age  VehicleID
0   3.272476e+08           1    10   0         41
1   6.630844e+08           2    10   0         41
2   9.677932e+07           3    10   0         41
3   8.784244e+06           9    10   0         41
0   3.272476e+08           1    11   0         41
..           ...         ...   ...  ..        ...
3   1.519609e+07           9     8   9         31
0   1.836098e+11           1     9   9         31
1   6.090473e+09           2     9   9         31
2   5.594270e+08           5     9   9         31
3   1.519609e+07           9     9   9         31

[5952 rows x 5 columns]

In [29]:
merged_df

emission  pollutantID  fuelTypeID  VehicleID speed age  \
0      6.087480e+12           91           1         41    10   0   
1      4.423975e+11           90           1         41    10   0   
2      6.479495e+05          117           1         41    10   0   
3      5.419722e+07            3           1         41    10   0   
4      1.119209e+06          116           1         41    10   0   
...             ...          ...         ...        ...   ...  ..   
33723  0.000000e+00           90           9         31     9   9   
33724  2.168481e+04          117           9         31     9   9   
33725  0.000000e+00            3           9         31     9   9   
33726  4.491149e+04          116           9         31     9   9   
33727  2.405309e+10           91           9         31     9   9   

           distance  emission_per_distance  
0      3.272476e+08           18602.062008  
1      3.272476e+08            1351.873928  
2      3.272476e+08               0.001980  
3      3.272476e+08               0.165615  
4      3.272476e+08               0.003420  
...             ...                    ...  
33723  1.519609e+07               0.000000  
33724  1.519609e+07               0.001427  
33725  1.519609e+07               0.000000  
33726  1.519609e+07               0.002955  
33727  1.519609e+07            1582.847619  

[33728 rows x 8 columns]

In [30]:
pivot_df = merged_df.pivot(index=['fuelTypeID', 'VehicleID', 'speed', 'age'], columns='pollutantID', values='emission_per_distance')

# Reset the index to convert the index columns back to regular columns
pivot_df.reset_index(inplace=True)
pivot_df.fillna(0, inplace=True)


In [31]:
pivot_df.rename(columns = {3: 'NOx',90: 'CO2',91: 'Total Energy',110: 'PM2.5 Total Exh',116: 'PM2.5 Brakewear',117: 'PM2.5 Tirewear'},index={'pollutantID': 'Index'}, inplace = True)

In [32]:
pivot_df

pollutantID  fuelTypeID  VehicleID speed age       NOx          CO2  \
0                     1          0     1   0  0.007529  1243.316896   
1                     1          0     1   1  0.010511  1287.124450   
2                     1          0     1  10  0.058114  1689.426286   
3                     1          0     1  11  0.058114  1738.083045   
4                     1          0     1  12  0.058114  1788.689299   
...                 ...        ...   ...  ..       ...          ...   
5947                  9         41     9   5  0.000000     0.000000   
5948                  9         41     9   6  0.000000     0.000000   
5949                  9         41     9   7  0.000000     0.000000   
5950                  9         41     9   8  0.000000     0.000000   
5951                  9         41     9   9  0.000000     0.000000   

pollutantID  Total Energy  PM2.5 Total Exh  PM2.5 Brakewear  PM2.5 Tirewear  
0            17108.295372         0.003143         0.041696        0.002542  
1            17711.095565         0.003261         0.041696        0.002542  
2            23246.854128         0.007614         0.041696        0.002542  
3            23916.380321         0.007316         0.041696        0.002542  
4            24612.733068         0.006772         0.041696        0.002542  
...                   ...              ...              ...             ...  
5947         13405.852844         0.000000         0.018085        0.005312  
5948         14068.420257         0.000000         0.018085        0.005312  
5949         13868.570530         0.000000         0.018085        0.005312  
5950         14944.068760         0.000000         0.018085        0.005312  
5951         14999.893152         0.000000         0.018085        0.005312  

[5952 rows x 10 columns]

In [34]:
pivot_df.to_csv('output3_washington.csv', mode='a', index=False)

In [ ]:
#substituting fuel id's with actual values
pivot_df['fuelTypeID'] = pivot_df['fuelTypeID'].replace(1, 'Gasoline')
pivot_df['fuelTypeID'] = pivot_df['fuelTypeID'].replace(2, 'Diesel Fuel')
pivot_df['fuelTypeID'] = pivot_df['fuelTypeID'].replace(3, 'Compressed Natural Gas (CNG)')
pivot_df['fuelTypeID'] = pivot_df['fuelTypeID'].replace(5, 'Ethanol')
pivot_df['fuelTypeID'] = pivot_df['fuelTypeID'].replace(9, 'Electricity')

#substituting vehicle id's with actual values
pivot_df['VehicleID'] = pivot_df['VehicleID'].replace(62, 'Combination Long Haul Truck')
pivot_df['VehicleID'] = pivot_df['VehicleID'].replace(61, 'Combination Short Haul Truck')
pivot_df['VehicleID'] = pivot_df['VehicleID'].replace(32, 'Light Commercial Truck')
pivot_df['VehicleID'] = pivot_df['VehicleID'].replace(54, 'Motor Home')
pivot_df['VehicleID'] = pivot_df['VehicleID'].replace(11, 'Motorcycle')
pivot_df['VehicleID'] = pivot_df['VehicleID'].replace(41, 'Other Buses')
pivot_df['VehicleID'] = pivot_df['VehicleID'].replace(21, 'Passenger Car')
pivot_df['VehicleID'] = pivot_df['VehicleID'].replace(31, 'Passenger Truck')
pivot_df['VehicleID'] = pivot_df['VehicleID'].replace(51, 'Refuse Truck')
pivot_df['VehicleID'] = pivot_df['VehicleID'].replace(43, 'School Bus')
pivot_df['VehicleID'] = pivot_df['VehicleID'].replace(53, 'Single Unit Long-haul Truck')
pivot_df['VehicleID'] = pivot_df['VehicleID'].replace(52, 'Single Unit Short-haul Truck')
pivot_df['VehicleID'] = pivot_df['VehicleID'].replace(42, 'Transit Bus')

In [ ]:
pivot_df

In [17]:
pivot_df.to_csv('output_graph2.csv', mode='a', index=False)